In [5]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Split set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier


# Path to image folders, 
data_path = fr'../Datasets/config'
# os.listdir(data_path)

def remove_white_background(pixels):
    newPixels = []
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0
        newPixels.append(pixel)
    
    return newPixels



def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]



def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehut_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y

def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehut_labels(src)
    
    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test

def get_youdens_index(predictions, Y):
    # Calculate true positive/negative and false positive/negative
    tp = sum((Y == predictions) * (Y == 1) * 1)
    tn = sum((Y == predictions) * (Y == 0) * 1)
    fp = sum((Y != predictions) * (Y == 0) * 1)
    fn = sum((Y != predictions) * (Y == 1) * 1)
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    
    result = sensitivity - (1 - specificity)
    # Put it in a dateframe for nicer visuals
    df = pd.DataFrame({'Youdens Index': result})
    pd.set_option('display.max_rows', 200)
    
    return df

In [ ]:
import time

# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, y_train, y_validation, y_test = process_files(data_path)
# Fit/train train-datasets and store prediction vectors in variables
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red Config HL Test
print("Red Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = RandomForestClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")

# Green Config HL Test
print("Green Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
# Blue Config HL Test
print("Blue Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")

In [ ]:
# default alpha=0.0001

print("Red Config Alpha Tests")
# Red Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")
    
print("Green Config Alpha Tests")
# Green Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
print("Blue Config Alpha Tests")
# Blue Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")    

In [6]:
rf_red =  RandomForestClassifier(random_state=1)
rf_red.fit(X_red_train, y_train)
vector_red = rf_red.predict(X_red_train)
vector_red_val = rf_red.predict(X_red_validation)

rf_green =  RandomForestClassifier(random_state=1)
rf_green.fit(X_blue_train, y_train)
vector_green = rf_green.predict(X_blue_train)
vector_green_val = rf_green.predict(X_blue_validation)

rf_blue =  RandomForestClassifier(random_state=1)
rf_blue.fit(X_blue_train, y_train)
vector_blue = rf_red.predict(X_blue_train)
vector_blue_val = rf_red.predict(X_blue_validation)


Starting...
Finished


In [7]:
from sklearn.metrics import classification_report,confusion_matrix
#print(confusion_matrix(mlpc_green.predict(X_green_test), y_test))
print("Red model")
print(classification_report(y_validation, rf_red.predict(X_red_validation)))
print("Green model")
print(classification_report(y_validation, rf_green.predict(X_green_validation)))
print("Blue model")
print(classification_report(y_validation, rf_blue.predict(X_blue_validation)))

Red model
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        74
           1       1.00      1.00      1.00        95
           2       1.00      1.00      1.00        98
           3       1.00      1.00      1.00        73
           4       1.00      1.00      1.00        91
           5       1.00      1.00      1.00       146

   micro avg       1.00      1.00      1.00       577
   macro avg       1.00      1.00      1.00       577
weighted avg       1.00      1.00      1.00       577
 samples avg       1.00      1.00      1.00       577

Green model
              precision    recall  f1-score   support

           0       1.00      0.85      0.92        74
           1       1.00      0.98      0.99        95
           2       1.00      1.00      1.00        98
           3       0.00      0.00      0.00        73
           4       0.00      0.00      0.00        91
           5       0.47      1.00      0.64       146

 

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
X_combined_train = np.column_stack((vector_red, vector_green, vector_blue))
X_combined_val = np.column_stack((vector_red_val, vector_green_val, vector_blue_val))

rf_comb = RandomForestClassifier(random_state=1)
rf_comb.fit(X_combined_train, y_train)

comb_validation = rf_comb.predict(X_combined_val)

print(f' VALIDATION PREDICTIONS: \n {classification_report(y_validation, comb_validation)} \n')

 VALIDATION PREDICTIONS: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        74
           1       1.00      1.00      1.00        95
           2       1.00      1.00      1.00        98
           3       1.00      1.00      1.00        73
           4       1.00      1.00      1.00        91
           5       1.00      1.00      1.00       146

   micro avg       1.00      1.00      1.00       577
   macro avg       1.00      1.00      1.00       577
weighted avg       1.00      1.00      1.00       577
 samples avg       1.00      1.00      1.00       577
 

